# Airports

In [55]:
# tabele wynikowe
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("databases/processed.sql")
conn           = sqlite3.connect("databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from airports", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("airports", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

# Mean departure delay

In [3]:
import sqlite3
import csv
conn = sqlite3.connect("databases/flights_full.sql")

conn.execute("""
CREATE TABLE US_filghts_dep_delay_Y AS 
    SELECT ontime.year                              AS year,
            origin.latitude_deg                     AS origin_lat, 
            origin.longitude_deg                    AS origin_lon, 
            origin.name                             AS origin_name, 
            ontime.Origin                           AS origin_id, 
            dest.latitude_deg                       AS dest_lat, 
            dest.longitude_deg                      AS dest_lon, 
            dest.name                               AS dest_name,
            ontime.Dest                             AS dest_id,
            COUNT(*)                                AS number_of_flights,
            SUM(ontime.DepDelay > 15)               AS delay_over_15,
            AVG(ontime.DepDelay)                    AS DepDelay, 
            AVG(ontime.ArrDelay - ontime.DepDelay)  AS delay_gain 
    FROM ontime 
    LEFT JOIN airports AS origin ON ontime.Origin == origin.iata
    LEFT JOIN airports AS dest   ON ontime.Dest   == dest.iata
    WHERE origin.country == 'US' AND dest.country == 'US' 
    GROUP BY ontime.year,
            ontime.Origin,
            ontime.Dest
""")
conn.commit()

In [2]:
#conn.execute("""
#DROP TABLE US_filghts_dep_delay_Y 
#""")
#conn.commit()

In [58]:
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("databases/processed.sql")
conn           = sqlite3.connect("databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from US_filghts_dep_delay_Y", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("US_filghts_dep_delay_Y", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

# Daily patterns


In [16]:
#conn.execute("""
#DROP TABLE Arr_Delays_vs_time_h
#""")
#conn.commit()

In [17]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("databases/flights_full.sql")

conn.execute("""
CREATE TABLE Arr_Delays_vs_time_h AS 
    SELECT  CASE 
               WHEN ontime.CRSDepTime == 2400 
                  THEN 0
                  ELSE ontime.CRSDepTime 
            END                                     AS time,
            ontime.UniqueCarrier                    AS UniqueCarrier, 
            COUNT(*)                                AS number_of_flights,
            AVG(ontime.DepDelay)                    AS DepDelay, 
            AVG(ontime.ArrDelay)                    AS ArrDelay 
    FROM ontime
    WHERE ontime.DepTime IS NOT NULL           AND
            typeof(ontime.DepTime) = "integer" AND
            ontime.DepTime % 100 < 60          AND
            ontime.DepTime < 2400
    GROUP BY time,
            ontime.UniqueCarrier
    HAVING DepDelay IS NOT NULL                AND
            ArrDelay IS NOT NULL
    ORDER BY  UniqueCarrier, time
""")
conn.commit()

In [60]:
# tabele wynikowe
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("databases/processed.sql")
conn           = sqlite3.connect("python/pd4/databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from Arr_Delays_vs_time_h", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("Arr_Delays_vs_time_h", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

# Weakly patterns

In [3]:
#conn.execute("""
#DROP TABLE Arr_Delays_vs_DayOfWeek
#""")
#conn.commit()

In [5]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("databases/flights_full.sql")

conn.execute("""
CREATE TABLE Arr_Delays_vs_DayOfWeek AS 
    SELECT  ontime.DayOfWeek                        AS day,
            ontime.UniqueCarrier                    AS UniqueCarrier, 
            COUNT(*)                                AS number_of_flights,
            AVG(ontime.ArrDelay)                    AS ArrDelay ,
            AVG(ontime.DepDelay)                    AS DepDelay 
    FROM ontime
    WHERE ontime.DayOfWeek IS NOT NULL           AND
            typeof(ontime.DayOfWeek) = "integer"
    GROUP BY ontime.DayOfWeek, ontime.UniqueCarrier
    HAVING ArrDelay IS NOT NULL
    ORDER BY  UniqueCarrier, day
""")
conn.commit()

In [61]:
# tabele wynikowe
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("databases/processed.sql")
conn           = sqlite3.connect("databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from Arr_Delays_vs_DayOfWeek", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("Arr_Delays_vs_DayOfWeek", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

# Yearly patterns

In [2]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("databases/flights_full.sql")

conn.execute("""
CREATE TABLE Delays_vs_Date AS 
    SELECT  date(year || '-' || substr('00'||month, -2, 2) || '-' || substr('00'||dayofmonth, -2, 2)) as date,
            ontime.year                             AS year,
            COUNT(*)                                AS number_of_flights,
            AVG(ontime.DepDelay)                    AS depDelay,
            SUM(ontime.DepDelay > 15)               AS delay_over_15,
            AVG(ontime.ArrDelay)                    AS arrDelay,
            SUM(ontime.ArrDelay > 15)               AS arrival_over_15
    FROM ontime
    WHERE date IS NOT NULL
    GROUP BY  date, year
    ORDER BY  date, year
""")
conn.commit()

In [62]:
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("databases/processed.sql")
conn           = sqlite3.connect("databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from Delays_vs_Date", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("Delays_vs_Date", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

# Correclation 

In [2]:
#conn.execute("""
#DROP TABLE Cor_Data2
#""")
#conn.commit()

In [3]:
from os.path import dirname, abspath
import pandas as pd
import sqlite3

conn = sqlite3.connect("databases/flights_full.sql")
origin = open("data/origin.txt").read().split()
it = 0

for o in origin:
    if it == 0:
        conn.execute("""
        CREATE TABLE Cor_Data2 AS 
            SELECT  date(year || '-' || substr('00'||month, -2, 2) || '-' || substr('00'||dayofmonth, -2, 2)) as date,
                    ontime.year                             AS year,
                    ontime.month                            AS month,
                    ontime.dayofmonth                       AS dayofmonth,
                    SUM(ontime.DepDelay > 15)               AS '%s'
            FROM ontime
            WHERE date IS NOT NULL AND ontime.origin == '%s'
            GROUP BY  date, year
            ORDER BY  date, year
        """ % (o,o))
        conn.commit()
        it+=1
        print(it)
    else:
        conn.execute("""
        ALTER TABLE Cor_Data2 ADD COLUMN %s integer;
        """ % (o,))
        conn.commit()
        conn.execute("""
        UPDATE Cor_Data2    
        SET %s = (
                  SELECT  SUM(ontime.DepDelay > 15)
                  FROM ontime
                  WHERE 
                        ontime.year == Cor_Data2.year             AND
                        ontime.month == Cor_Data2.month           AND
                        ontime.dayofmonth == Cor_Data2.dayofmonth AND                 
                        ontime.origin == '%s'
                  GROUP BY  ontime.year, ontime.month, ontime.dayofmonth
                  ORDER BY  ontime.year, ontime.month, ontime.dayofmonth
                  )
        """ % (o,o,))
        conn.commit()
        print(it)
        it+=1
        

1
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44


In [63]:
import sqlite3
import pandas as pd

conn_processed = sqlite3.connect("databases/processed.sql")
conn           = sqlite3.connect("databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from Cor_Data2", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("timeDep", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

# Producents

In [12]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("databases/flights_full.sql")

conn.execute("""
CREATE TABLE Tailnums AS 
    SELECT  ontime.Tailnum                          AS tailnum
    FROM ontime
    WHERE ontime.year<>2002 AND ontime.year<>2001
    GROUP BY  tailnum
""")
conn.commit()

In [11]:
#conn.execute("""
#DROP TABLE Tailnums
#""")
#conn.commit()

In [86]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("databases/flights_full.sql")

cur = conn.execute("""
SELECT tailnum from Tailnums
""")

tailnum = cur.fetchall()

In [92]:
import requests
import pandas as pd
import time

tails = []
manufacturers = []
models = []
owners = []

tails_problem = []

for tail in tailnum:
    tail = tail[0]
    if len(tail) == 6 and tail.startswith("N"):
        url = 'http://registry.faa.gov/aircraftinquiry/NNum_Results.aspx?nNumberTxt=' + str(tail)
        html = requests.get(url).content
        df_list = pd.read_html(html)
        if len(df_list)<6:
            print("No data for tailnum: " + tail)
        else:
            try:
                df = df_list[4]
                manufacturer = df.iloc[1,1]
                model = df.iloc[2,1]
                df = df_list[5]
                owner = df.iloc[0,1]
                print(str(tail) + ' ' + str(manufacturer) + ' ' + str(model) + ' ' + str(owner))
                tails.append(tail)
                manufacturers.append(manufacturer)
                models.append(model)
                owners.append(owner)
            except Exception:
                print("Error")
                tails_problem.append(tail)
                
        time.sleep(0.2)

No data for tailnum: N001BR
No data for tailnum: N021BR
No data for tailnum: N043BR
No data for tailnum: N050AA
No data for tailnum: N050BR
No data for tailnum: N051AA
No data for tailnum: N052AA
No data for tailnum: N054AA
No data for tailnum: N055AA
No data for tailnum: N055BR
No data for tailnum: N056AA
No data for tailnum: N057AA
No data for tailnum: N058AA
No data for tailnum: N059AA
No data for tailnum: N060AA
No data for tailnum: N061AA
No data for tailnum: N062AA
No data for tailnum: N063AA
No data for tailnum: N064AA
No data for tailnum: N065AA
No data for tailnum: N066AA
No data for tailnum: N067AA
No data for tailnum: N068AA
No data for tailnum: N069AA
No data for tailnum: N070AA
No data for tailnum: N071AA
No data for tailnum: N072AA
No data for tailnum: N073AA
No data for tailnum: N074AA
No data for tailnum: N075AA
No data for tailnum: N075BR
No data for tailnum: N076AA
No data for tailnum: N077AA
No data for tailnum: N078AA
No data for tailnum: N079AA
No data for tailnum:

N123UW AIRBUS INDUSTRIE A320-214 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N124DE BOEING 767-332 DELTA AIR LINES INC
N124US AIRBUS INDUSTRIE A320-214 WELLS FARGO BANK NORTHWEST NA TRUSTEE
No data for tailnum: N124UW
N12519 EMBRAER EMB-135LR UNITED AIRLINES INC
N12528 EMBRAER EMB-135LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N12530 EMBRAER EMB-135LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N12540 EMBRAER EMB-145LR UNITED AIRLINES INC
N12552 EMBRAER EMB-145LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N12563 EMBRAER EMB-145LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N12564 EMBRAER EMB-145LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N12567 EMBRAER EMB-145LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N12569 EMBRAER EMB-145LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N125DL BOEING 767-332 DELTA AIR LINES INC
N126AT SEA & SKY INC CYGNET FLYING DOLPHINS LLC
N126DL BOEING 767-332 DELTA MATERIAL SERVICES LLC
N127DL BOEING 767-332 DELTA AIR LINES INC
N127UA BOEING 747-422 UNITED AIRLINES INC
N12811 nan nan nan
N128DL 5007070

N14904 EMBRAER EMB-145LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N14905 EMBRAER EMB-145LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N14907 EMBRAER EMB-145LR WELLS FARGO BANK NORTHWEST NA TRUSTEE
N14916 EMBRAER EMB-145LR UNITED AIRLINES INC
N14920 EMBRAER EMB-145LR SOUTHERN AIRCRAFT CONSULTANCY INC TRUSTEE
N14923 EMBRAER EMB-145LR SOUTHERN AIRCRAFT CONSULTANCY INC TRUSTEE
N14925 EMBRAER EMB-145EP AVI SALES & LEASING SERVICES LLC
N14930 EMBRAER EMB-145EP AAR SUPPLY CHAIN INC
N14933 EMBRAER EMB-145EP SPECTRE AIR CAPITAL LLC
N14937 50143400 10/04/2016 nan
N14938 50143401 03/15/2016 nan
N14939 50143402 08/13/2015 nan
N14940 50143434 08/13/2015 nan
N14942 50143436 11/15/2016 nan
N14943 50143437 05/26/2015 nan
N14945 EMBRAER EMB-145EP AEROVISION AIRCRAFT SERVICES LLC
N14947 EMBRAER EMB-145EP AEROVISION AIRCRAFT SERVICES LLC
No data for tailnum: N14950
N14952 50143501 10/05/2015 nan
N14953 50143502 02/03/2016 nan
N14959 EMBRAER EMB-145LR AEROVISION AIRCRAFT SERVICES LLC
N14960 50143542 08/06/2015 na

N17309 nan nan nan
N17316 nan nan nan
N17317 nan nan nan
N17321 nan nan nan
N17326 nan nan nan
N17328 nan nan nan
N17329 50223422 03/21/2013 nan
N17337 50223463 03/21/2017 nan
N17344 50223523 03/21/2013 nan
N17345 nan nan nan
N17356 50223570 03/21/2013 nan
N17358 50223572 05/08/2013 nan
N173AT AIRTIME AIRCRAFT INC CYGNET AIRTIME AIRCRAFT INC
N173AW 50222147 03/13/2014 nan
N173DN 50222252 09/06/2017 nan
N173DZ BOEING 767-332 DELTA AIR LINES INC
N173UA 50223025 08/17/2017 nan
N173US AIRBUS INDUSTRIE A321-211 WELLS FARGO BANK NORTHWEST NA TRUSTEE
No data for tailnum: N173UW
No data for tailnum: N174AT
N174AW nan nan nan
N174DN BOEING 767-332 DELTA AIR LINES INC
N174DZ BOEING 767-332 DELTA AIR LINES INC
No data for tailnum: N174MQ
N174UA BOEING 747-422 WILMINGTON TRUST CO TRUSTEE
N174US AIRBUS INDUSTRIE A321-211 WELLS FARGO BANK NORTHWEST NA TRUSTEE
No data for tailnum: N174UW
N17507 EMBRAER EMB-135ER UNITED AIRLINES INC
N17513 EMBRAER EMB-135LR AVI SALES AND LEASING SERVICES LLC
N17521 EM

N202UA 50315464 12/08/2014 nan
N202US nan nan nan
N202UW BOEING 757-2B7 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N202WN BOEING 737-7H4 SOUTHWEST AIRLINES CO
N203AA CANADAIR LTD CL-600-2B16 CITE AVIATION LLC
N203JB EMBRAER ERJ 190-100 IGW WELLS FARGO BANK NORTHWEST NA TRUSTEE
N203UA 50317353 03/18/2013 nan
N203UW BOEING 757-23N AMERICAN AIRLINES INC
N203WN BOEING 737-7H4 SOUTHWEST AIRLINES CO
N204UA BOEING 777-222 UNITED AIRLINES INC
N204US BELL 206L-3 DAKOTA TERRITORY TOURS ACC
N204UW BOEING 757-23N AMERICAN AIRLINES INC
N204WN BOEING 737-7H4 SOUTHWEST AIRLINES CO
N205AA CESSNA 210-5(205) SKY LIFT AVIATION LLC
N205UA 50323131 12/08/2014 nan
N205UW BOEING 757-23N AMERICAN AIRLINES INC
N205WN BOEING 737-7H4 SOUTHWEST AIRLINES CO
N206JB EMBRAER ERJ 190-100 IGW WELLS FARGO BANK NORTHWEST NA TRUSTEE
N206UA BOEING 777-222 UNITED AIRLINES INC
N206UW BOEING 757-2B7 AMERICAN AIRLINES INC
N206WN BOEING 737-7H4 SOUTHWEST AIRLINES CO
N207AA GULFSTREAM AEROSPACE G-IV AMB GROUP LLC
N207SW nan nan nan
N

...

N8908E 53044311 08/08/2013 nan
N8909E 53044354 03/11/2013 nan
N8911E 53045613 08/16/2013 nan
N8912E 53045656 08/06/2013 nan
N8913E 53045721 08/15/2013 nan
N8914E 53045764 03/11/2013 nan
N8915E 53046027 08/27/2013 nan
N891AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N8920E 53047437 11/19/2013 nan
N8921E 53047502 11/19/2013 nan
N8923E nan nan nan
N8925E nan nan nan
N8926E nan nan nan
N8928E nan nan nan
N8929E 53050132 11/19/2013 nan
N892AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N8932E 53051434 08/21/2013 nan
N8933E nan nan nan
N8934E nan nan nan
N8938E 53051756 11/19/2013 nan
N893AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N8944E 53053431 08/27/2013 nan
N8945E nan nan nan
N894AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N8950E nan nan nan
N8957E nan nan nan
N895AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N8960E 53056773 06/10/2013 nan
N896AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N8978E nan nan nan
N8979E nan nan n

N912EV BOMBARDIER INC CL-600-2B19 EXPRESSJET AIRLINES INC
N912FJ BOMBARDIER INC CL-600-2D24 MESA AIRLINES INC
N912FR 53117246 02/05/2016 nan
N912MD DUENSING M J RV-4 OWENS DOUGLAS P
N912SW BOMBARDIER INC CL-600-2B19 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N912UA 53117734 03/18/2013 nan
N912WN BOEING 737-7H4 SOUTHWEST AIRLINES CO
N913AW nan nan nan
N913CA nan nan nan
N913DE MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N913DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N913DN MCDONNELL DOUGLAS MD-90-30 DELTA AIR LINES INC
N913EV BOMBARDIER INC CL-600-2B19 EXPRESSJET AIRLINES INC
N913FJ BOMBARDIER INC CL-600-2D24 MESA AIRLINES INC
N913FR nan nan nan
N913SW BOMBARDIER INC CL-600-2B19 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N913UA 53121623 03/21/2013 nan
N913WN BOEING 737-7H4 SOUTHWEST AIRLINES CO
N914AW 53122634 08/21/2013 nan
N914CA CANADAIR CL-600-2B19 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N914DE MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N914DL MCDONNELL 

N927RC 53154645 02/07/2013 nan
N927SW BOMBARDIER INC CL-600-2B19 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N927UA 53154756 03/21/2013 nan
N927UW AIRBUS A321-231 AMERICAN AIRLINES INC
N927WN BOEING 737-7H4 SOUTHWEST AIRLINES CO
N92874 53157514 03/21/2013 nan
N928AE EMBRAER EMB-145LR AMERICAN AIRLINES INC
N928AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N928DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N928EV BOMBARDIER INC CL-600-2B19 SKYWEST AIRLINES INC
N928FR 53156157 05/22/2017 nan
N928LR BOMBARDIER INC CL-600-2D24 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N928MQ nan nan nan
N928SW BOMBARDIER INC CL-600-2B19 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N928UA 53156645 11/27/2017 nan
No data for tailnum: N928UW
N928WN BOEING 737-7H4 SOUTHWEST AIRLINES CO
N929AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N929CA 53157714 11/04/2013 nan
N929DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N929EV BOMBARDIER INC CL-600-2B19 SKYWEST AIRLINES INC
N929FR 53160046

No data for tailnum: N947UA
N947UW EMBRAER ERJ 190-100 IGW AMERICAN AIRLINES INC
No data for tailnum: N947WP
N948AS 53225361 09/15/2017 nan
N948AT BOEING 717-200 AIRTRAN AIRWAYS INC
N948DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N948FR AIRBUS A319-112 AFS INVESTMENTS 59 LLC
N948SW BOMBARDIER INC CL-600-2B19 WELLS FARGO BANK NORTHWEST NA TRUSTEE
No data for tailnum: N948UA
N948UW EMBRAER ERJ 190-100 IGW AMERICAN AIRLINES INC
N948WP nan nan nan
N949AS BOEING 747-45E AERSALE INC
N949AT BOEING 717-200 AIRTRAN AIRWAYS INC
N949CA 53227312 03/26/2014 nan
N949DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N949FR AIRBUS A319-112 FRONTIER AIRLINES INC
N949UA nan nan nan
N949UW EMBRAER ERJ 190-100 IGW AMERICAN AIRLINES INC
N950AS 53232270 06/21/2017 nan
N950AT BOEING 717-200 AIRTRAN AIRWAYS INC
N950DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N950FR nan nan nan
N950UA UNIV OF ALABAMA AT HUNTSVILLE PEGASUS UNIVERSITY OF ALABAMA IN HUNTSVILLE
N950UW EMBRAER

N975AS 53313111 07/07/2015 nan
N975AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N975CA SOCATA TBM 700 DELALIO MANAGEMENT LLC
N975DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 WILMINGTON TRUST CO - OWNER TRUSTEE
N975EV BOMBARDIER INC CL-600-2B19 US BANK NA TRUSTEE
N975SW BOMBARDIER INC CL-600-2B19 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N976AS 53315000 03/30/2015 nan
N976CA BEECH 76 ELON AVIATION LLC
N976DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N976EV BOMBARDIER INC CL-600-2B19 U S BANK NA TRUSTEE
N976SW BOMBARDIER INC CL-600-2B19 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N977AS 53316667 06/17/2014 nan
N977AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N977CA nan nan nan
N977DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N977EV BOMBARDIER INC CL-600-2B19 US BANK NA TRUSTEE
N978AT BOEING 717-200 WELLS FARGO BANK NORTHWEST NA TRUSTEE
N978CA 53320620 11/27/2017 nan
N978DL MCDONNELL DOUGLAS AIRCRAFT CO MD-88 DELTA AIR LINES INC
N978EV BOMBARDIER INC CL-6

In [97]:
planes = pd.DataFrame(
    {'tails': tails,
     'manufacturers': manufacturers,
     'models': models,
     'owners': owners
    })

                      manufacturers          models  \
0                           EMBRAER       EMB-145XR   
1         DIAMOND AIRCRAFT IND GMBH           DA 42   
2                               NaN             NaN   
3                            BOEING         767-232   
4                  AIRBUS INDUSTRIE        A320-214   
5                               NaN             NaN   
6        AIRCRAFT SALES & PARTS LTD       SUMMIT II   
7                               NaN             NaN   
8                  AIRBUS INDUSTRIE        A320-214   
9                    BOMBARDIER INC     BD-700-1A10   
10               FLIGHT DESIGN GMBH            CTSW   
11                           BOEING         747-422   
12                 AIRBUS INDUSTRIE        A320-214   
13                          EMBRAER       EMB-145LR   
14                  AIR TRACTOR INC          AT-301   
15                         50014354      06/07/2013   
16                           BOEING         747-451   
17        

In [9]:
planes_processed = planes.dropna()

In [108]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("databases/flights_full.sql")


# Write the new DataFrame to a new SQLite table
planes_processed.to_sql("planes", conn, if_exists="replace")

conn.close()

In [109]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("databases/flights_full.sql")


# Write the new DataFrame to a new SQLite table
planes.to_sql("planes_raw", conn, if_exists="replace")

conn.close()

# Model Performance 

In [20]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

conn.execute("""
CREATE TABLE Model_Performance AS 
    SELECT  ontime.year                             AS year,
            ontime.month                            AS month,
            AVG(ontime.DepDelay)                    AS dep_delay,
            SUM(ontime.DepDelay > 15)               AS delay_over_15,
            COUNT(*)                                AS number_of_flights,
            planes.manufacturers                    AS manufacturer,
            planes.models                           AS model            
    FROM ontime
    LEFT JOIN planes ON ontime.Tailnum == planes.tails
    WHERE ontime.year > 2002                   AND 
          planes.manufacturers IS NOT NULL     AND
          planes.models IS NOT NULL
    GROUP BY  ontime.year, ontime.month, planes.manufacturers, planes.models
    UNION
    SELECT  ontime.year                             AS year,
            ontime.month                            AS month,
            AVG(ontime.DepDelay)                    AS dep_delay,
            SUM(ontime.DepDelay > 15)               AS delay_over_15,
            COUNT(*)                                AS number_of_flights,
            planes.manufacturers                    AS manufacturer,
            "ALL"                                   AS model          
    FROM ontime
    LEFT JOIN planes ON ontime.Tailnum == planes.tails
    WHERE ontime.year > 2002                   AND 
          planes.manufacturers IS NOT NULL     AND
          planes.models IS NOT NULL
    GROUP BY  ontime.year, ontime.month, planes.manufacturers
    UNION
    SELECT  ontime.year                             AS year,
            ontime.month                            AS month,
            AVG(ontime.DepDelay)                    AS dep_delay,
            SUM(ontime.DepDelay > 15)               AS delay_over_15,
            COUNT(*)                                AS number_of_flights,
            "ALL"                                   AS manufacturer,
            "ALL"                                   AS model  
    FROM ontime
    LEFT JOIN planes ON ontime.Tailnum == planes.tails
    WHERE ontime.year > 2002                   AND 
          planes.manufacturers IS NOT NULL     AND
          planes.models IS NOT NULL
    GROUP BY  ontime.year, ontime.month
""")
conn.commit()

In [37]:
conn.execute("""
CREATE TABLE Model_Names AS 
    SELECT  manufacturer                    AS manufacturer,
            model                           AS model
    FROM Model_Performance
    GROUP BY manufacturer, model
    ORDER BY manufacturer asc , SUM(number_of_flights) desc
""")
conn.commit()

In [45]:
conn.execute("""
CREATE TABLE Manufacturer_Names AS 
    SELECT  manufacturer                    AS manufacturer
    FROM Model_Performance
    GROUP BY manufacturer
    ORDER BY SUM(number_of_flights) desc
""")
conn.commit()

In [36]:
#conn.execute("""
#DROP TABLE Model_Performance
#""")
#conn.commit()
#conn.execute("""
#DROP TABLE Model_Names
#""")
#conn.commit()

In [51]:
import sqlite3
import pandas as pd

conn_processed = sqlite3.connect("/home/pkosewski/python/pd4/databases/processed.sql")
conn           = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from Model_Performance", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("Model_Performance", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

In [52]:
import sqlite3
import pandas as pd

conn_processed = sqlite3.connect("/home/pkosewski/python/pd4/databases/processed.sql")
conn           = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from Model_Names", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("Model_Names", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

In [53]:
import sqlite3
import pandas as pd

conn_processed = sqlite3.connect("/home/pkosewski/python/pd4/databases/processed.sql")
conn           = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from Manufacturer_Names", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("Manufacturer_Names", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()